In [2]:
import dlib, cv2
import numpy as np
import matplotlib.pyplot as plt

detector = dlib.get_frontal_face_detector() # dlib에 있는 정면 얼굴 검출기 사용
sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

In [3]:
# 얼굴 탐지
def find_faces(img):
    dets = detector(img, 1)
    
    if len(dets) == 0:
        return np.empty(0), np.empty(0), np.empty(0)
    
    rects, shapes = [], []
    shapes_np = np.zeros((len(dets), 68, 2), dtype=np.int)
    for k, d in enumerate(dets):
        rect = ((d.left(), d.top()), (d.right(), d.bottom()))
        rects.append(rect)

        shape = sp(img, d)
        
        for i in range(0, 68):
            shapes_np[k][i] = (shape.part(i).x, shape.part(i).y)

        shapes.append(shape)
        
    return rects, shapes, shapes_np

# 랜드마크 추출
def encode_faces(img, shapes):
    face_descriptors = []
    for shape in shapes:
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        face_descriptors.append(np.array(face_descriptor))

    return np.array(face_descriptors)

In [4]:
img_paths = {
    '나':'Me.jpg',
}

descs = []

for name, img_path in img_paths.items(): 
    img = cv2.imread(img_path)
    _, img_shapes, _ = find_faces(img)
    descs.append([name, encode_faces(img, img_shapes)[0]])
    
np.save('descs.npy', descs)
print(descs)

[['나', array([-0.09853306,  0.04192254, -0.00750632, -0.02881538, -0.11943647,
       -0.05633156, -0.05924125, -0.14445364,  0.13263528, -0.08700871,
        0.23490448, -0.09340746, -0.16063353, -0.09778604, -0.09571584,
        0.17909861, -0.13252419, -0.13934724, -0.04285906, -0.00574602,
        0.07480837,  0.04828412, -0.02839876,  0.03675881, -0.06291895,
       -0.27205551, -0.09402128, -0.10892911,  0.07490613, -0.08709935,
       -0.06554682,  0.05368298, -0.17564066, -0.11641715,  0.09280898,
        0.11721478, -0.04752152, -0.0395039 ,  0.16571058, -0.02204025,
       -0.24184258,  0.0734133 ,  0.06808091,  0.25764304,  0.18676937,
        0.03409481, -0.0016872 , -0.22008914,  0.14190941, -0.11262579,
        0.04184308,  0.14425655,  0.14679989,  0.08105846, -0.01558022,
       -0.11328605,  0.04615614,  0.11244515, -0.15493034,  0.04187237,
        0.16165553, -0.07144943, -0.01084448, -0.08212176,  0.17216527,
       -0.01532686, -0.1253121 , -0.18197767,  0.10488976

C:\Users\user\anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [5]:
cam = cv2.VideoCapture(0) # 노트북 웹캠 사용

while True:
    
    ret, img = cam.read() 
    img = cv2.flip(img, 1) # 좌우 대칭
    rects, shapes, _ = find_faces(img) # 얼굴 찾기
    descriptors = encode_faces(img, shapes) # 인코딩
    
    for i, desc in enumerate(descriptors):
        x = rects[i][0][0] # 얼굴 X 좌표
        y = rects[i][0][1] # 얼굴 Y 좌표
        w = rects[i][1][1]-rects[i][0][1] # 얼굴 너비 
        h = rects[i][1][0]-rects[i][0][0] # 얼굴 높이
        
        # 추출된 랜드마크와 데이터베이스의 랜드마크들 중 제일 짧은 거리를 찾는 부분
        descs1 = sorted(descs, key=lambda x: np.linalg.norm([desc] - x[1]))
        dist = np.linalg.norm([desc] - descs1[0][1], axis=1)
        
        if dist < 0.5: # 그 거리가 0.5보다 작다면 그 사람으로 판단 
            name = descs1[0][0]
        else:           # 0.5보다 크다면 모르는 사람으로 판단 -> 모자이크 처리
            name = "모르는사람"
            mosaic_img = cv2.resize(img[y:y+h, x:x+w], dsize=(0, 0), fx=0.04, fy=0.04) # 축소
            mosaic_img = cv2.resize(mosaic_img, (w, h), interpolation=cv2.INTER_AREA)  # 확대
            img[y:y+h, x:x+w] = mosaic_img # 인식된 얼굴 영역 모자이크 처리 
      
    cv2.imshow('camera', img)
    
    k = cv2.waitKey(10) & 0xff # 'ESC' 키 누르면 종료
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()